# Подготовка текста (6 баллов)
В первой части нужно подготовить данные

In [1]:
!pip install --quiet sentencepiece datasets transformers


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from tqdm.notebook import tqdm, trange
from tokenizers.decoders import WordPiece
from torch.utils.data import DataLoader
from dataset_maker import DatasetMaker, collate_batch, PAD_TOKEN_IDX
from utils import clean_text, read_file

decoder = WordPiece()

In [3]:
read_file('./data/books_txt/AKSAKOW/bagrov.txt')

'Сергей Тимофеевич Аксаков. Детские годы Багрова-внука\n\n\nСергей Тимофеевич Аксаков. Детские годы Багрова-внука\n\n\n\nСергей Тимофеевич Аксаков\nДетские годы Багрова-внука\n\n\nАксаков С.Т. Детские годы Багрова-внука. (Для сред. школ. возраста)\nМн., Юнацтва, 1982. - 318 с., ил.\nOCR & SpellCheck: Zmiy (zmiy@inbox.ru), 24 ноября 2003 года\n\n\n\n\n\n\n\n     -----------------------------------------------------------------------\n     Аксаков С.Т. Детские годы Багрова-внука. (Для сред. школ. возраста)\n     Мн., Юнацтва, 1982. - 318 с., ил.\n     OCR & SpellCheck: Zmiy (zmiy@inbox.ru), 24 ноября 2003 года\n     -----------------------------------------------------------------------\n\n     В автобиографической книге автор  показывает  дворянско-крепостническую\nсреду, в которой формировался характер Сережи  Багрова,  раскрывает  влияние\nна мальчика родной природы, общения с ней.\n\n\n\nСОДЕРЖАНИЕ\n\n\n                  К читателям\n\n                  Вступление\n                  

In [4]:
clean_text(read_file('./data/books_txt/AKSAKOW/bagrov.txt'))

'сергей тимофеевич аксаков. детские годы багрова-внука сергей тимофеевич аксаков. детские годы багрова-внука сергей тимофеевич аксаков детские годы багрова-внука аксаков с.т. детские годы багрова-внука. (для сред. школ. возраста) мн., юнацтва, 1982. - 318 с., ил. ocr spellcheck: zmiy (zmiy inbox.ru), 24 ноября 2003 года ----------------------------------------------------------------------- аксаков с.т. детские годы багрова-внука. (для сред. школ. возраста) мн., юнацтва, 1982. - 318 с., ил. ocr spellcheck: zmiy (zmiy inbox.ru), 24 ноября 2003 года ----------------------------------------------------------------------- в автобиографической книге автор показывает дворянско-крепостническую среду, в которой формировался характер сережи багрова, раскрывает влияние на мальчика родной природы, общения с ней. содержание к читателям вступление отрывочные воспоминания последовательные воспоминания дорога до парашина парашино дорога из парашина в багрово багрово пребывание в багрове без отца и ма

In [5]:
DIRECTORY = './data/books_txt'
VOCAB_PATH = './data/vocab.txt'
MAX_SEQ_LEN = 256
BATCH_SIZE = 16
OFFSET = MAX_SEQ_LEN // 2
TEST_SIZE = 0.03

In [6]:
dataset_maker = DatasetMaker(directory=DIRECTORY, test_size=TEST_SIZE, vocab_path=VOCAB_PATH, max_seq_len=MAX_SEQ_LEN, batch_size=BATCH_SIZE, offset=OFFSET)

In [7]:
texts_train, texts_test = dataset_maker.get_texts()
print(len(texts_train), len(texts_test))

726 23


Для токенизации используется готовое решение из библиотеки Huggingface

In [8]:
ids = dataset_maker.encode("я помню чудное мгновенье")
ids

[156, 2769, 2723, 539, 2646, 1881]

In [9]:
dataset_maker.tokenizer.convert_ids_to_tokens(ids)

['я', 'помню', 'чуд', '##ное', 'мгнов', '##енье']

In [10]:
decoder.decode(dataset_maker.tokenizer.convert_ids_to_tokens(ids))

'я помню чудное мгновенье'

- Конкатенируйте все тексты в большую последовательность токенов
- Посмотрим количество токенов, должно получиться примерно 35087577

In [11]:
dataset_maker.combine_texts()
print(len(dataset_maker.tokenized_train))

35054756


In [12]:
print(dataset_maker.tokenized_train[:100])
decoded_tokens = decoder.decode(dataset_maker.tokenizer.convert_ids_to_tokens(dataset_maker.tokenized_train[:100]))
print(decoded_tokens)

[4578, 3506, 18, 950, 127, 5628, 212, 133, 7466, 6447, 4578, 3506, 18, 950, 127, 5628, 212, 133, 7466, 6447, 125, 18, 140, 18, 3506, 18, 6336, 6919, 2707, 772, 133, 3122, 127, 4232, 17, 839, 838, 4727, 18, 6146, 18, 838, 21, 18, 137, 18, 16, 6, 7098, 6, 16, 943, 247, 241, 4133, 213, 5942, 244, 17, 3700, 3021, 3341, 184, 820, 3697, 342, 50, 221, 221, 203, 30, 5616, 210, 7654, 4819, 18, 7854, 7150, 4114, 673, 5444, 220, 225, 1744, 7444, 200, 228, 18, 50, 221, 226, 214, 1131, 518, 18, 1713, 1455, 1111, 16, 3251]
антон чехов. жизнь в вопросах и восклицаниях антон чехов. жизнь в вопросах и восклицаниях а. п. чехов. полное собрание сочинении и писем в 30 - ти томах. сочинения. том 1. м., " наука ", 1983 ocr 1996 - 2000 алексеи комаров http : ilibrary. ru author chekhov index. html детство. кого бог дал, сына


Далее нужно нарезать текст на части размером `max_seq_len + 1`. Полезно что бы части пересекались, для этого добавлен параметр `offset`

In [13]:
train_dataloader = DataLoader(dataset_maker, BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

In [14]:
print(len(dataset_maker))

272773


In [16]:
batch = next(iter(train_dataloader))
print(batch.shape)

torch.Size([16, 257])
